# Capstone Project - The Battle of Neighborhoods (Week 2)
## Project Title: Where should I open a bakery in NE Ohio?

### Background

Question: What location around NorthEast Ohio will be a great place to open a bakery?

#### Introduction/Business Problem:
During a casual conversation, a fellow baker friend asked me which location would be best to open a bakery nearby? I do not know the answer but we started to wonder if having certain businessess and the population around the area would influence the business model. Finding out if there is a business type patterns of successful bakeries that we know of around the nearby neighborhoods, we may be able to assess other areas without bakeries to be suitable to open a bakery

##### Data:
Locating a place of business interest:
1. List the bakeries we think are successful that we want to mimick
2. Find the neighborhoods of the successful bakeries above through FourSquare
3. Obtain data around those neighborhoods and the neighborhoods we are interested in opening businesses
4. Sort the top 10 businesses around those neighborhoods and use knn method to assess if there is a similar patterns between the successful bakeries and also neighborhoods of interest without bakeries

Hypothesizing the potential revenue:
1. Obtain the successful bakeries' demographic information. Planning obtain data from https://www.census.gov/quickfacts/fact/table/OH/PST045219
2. Obtain the demographic of neighborhoods of interest
3. Hypothesize an estimate of proportion of age group and frequency of customers in the successful bakery, compared to its baseline demographics. 
4. Then use this information to reflect on the top 5 neighborhood of business interest, and calculate through its population and demographics how much potential revenue we can generate in the first year. Assuming there is no seasonality to the consumers' purchasing behavior. 


In [75]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
import numpy as np
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [76]:
url = 'https://raw.githubusercontent.com/micceee/bakery/60bb9235113e68b0fd4ed917af3964fa124cf6c0/lat_long_ohio_city.csv'
ohio_list = pd.read_csv(url)

ohio_list.head()

,city,lat,lon
0,Akron,41.083064,-81.518485
1,Alliance,40.915336,-81.105931
2,Amherst,41.397822,-82.222377
3,Ashland,40.868615,-82.317594
4,Ashtabula,41.865054,-80.789809


In [77]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
ACCESS_TOKEN = ''
VERSION = '' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500
neighborhood_latitude = ohio_list.lat[[151]][151]
neighborhood_longitude = ohio_list.lon[[151]][151]




In [78]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [79]:
# get json from 4square API
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '612ab89748c9d119908f00dd'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'North Ridgeville',
  'headerFullLocation': 'North Ridgeville',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 41.3939905045, 'lng': -82.01304515152137},
   'sw': {'lat': 41.3849904955, 'lng': -82.02501904847864}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b672f6af964a52010402be3',
       'name': 'Olde Towne Hall Theatre',
       'location': {'address': '36119 Center Ridge Rd',
        'lat': 41.38929407770012,
        'lng': -82.02009726131836,
        'labeledLatLngs': [{'label': 

In [80]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [81]:
# clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Olde Towne Hall Theatre,Theater,41.389294,-82.020097
1,BP,Gas Station,41.389895,-82.018266
2,Mr. Tire Auto Service Centers,Automotive Shop,41.390311,-82.020211


In [82]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [83]:
ohio_venues = getNearbyVenues(names=ohio_list['city'],
                                   latitudes=ohio_list['lat'],
                                   longitudes=ohio_list['lon']
                                  )

Akron
Alliance
Amherst
Ashland
Ashtabula
Athens
Aurora
Avon
Avon+Lake
Barberton
Bay+Village
Beachwood
Beavercreek
Bedford
Bedford+Heights
Bellbrook
Bellefontaine
Bellevue
Belpre
Berea
Bexley
Blue+Ash
Bowling+Green
Brecksville
Broadview+Heights
Brooklyn
Brook+Park
Brookville
Brunswick
Bryan
Bucyrus
Cambridge
Campbell
Canal+Fulton
Canal+Winchester
Canfield
Canton
Celina
Centerville
Chardon
Cheviot
Chillicothe
Cincinnati
Circleville
Clayton
Cleveland
Cleveland+Heights
Clyde
Columbiana
Columbus
Conneaut
Cortland
Coshocton
Cuyahoga+Falls
Dayton
Deer+Park
Defiance
Delaware
Delphos
Dover
Dublin
East+Cleveland
East+Liverpool
Eastlake
Eaton
Elyria
Englewood
Euclid
Fairborn
Fairfield
Fairlawn
Fairview+Park
Findlay
Forest+Park
Fostoria
Franklin
Fremont
Gahanna
Galion
Garfield+Heights
Geneva
Germantown
Girard
Grandview+Heights
Green
Greenville
Grove+City
Groveport
Hamilton
Harrison
Heath
Highland+Heights
Hilliard
Hillsboro
Hubbard
Huber+Heights
Hudson
Huron
Independence
Ironton
Jackson
Kent
Kenton

In [73]:
print(ohio_venues.shape)
ohio_venues

pd.DataFrame.to_csv(ohio_venues, "ohio_venues.csv")

(3689, 7)


In [ ]:
ohio_venues = pd.read_csv("ohio_home_venues.csv")

# one hot encoding
ohio_onehot = pd.get_dummies(ohio_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ohio_onehot['Neighborhood'] = ohio_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ohio_onehot.columns[-1]] + list(ohio_onehot.columns[:-1])
ohio_onehot = ohio_onehot[fixed_columns]

ohio_onehot

In [ ]:
ohio_grouped = ohio_onehot.groupby('Neighborhood').mean().reset_index()
ohio_grouped

In [ ]:
num_top_venues = 10

for hood in ohio_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ohio_grouped[ohio_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ohio_grouped['Neighborhood']

for ind in np.arange(ohio_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ohio_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

neighborhoods_venues_sorted = neighborhoods_venues_sorted.rename(columns={'Neighborhood':'city'})

In [ ]:
# set number of clusters
kclusters = 5

ohio_grouped_clustering = ohio_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ohio_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

In [ ]:
# add clustering labels
# neighborhoods_venues_sorted.drop('Cluster Labels',1)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ohio_merged = ohio_list

# # merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
ohio_merged = ohio_merged.join(neighborhoods_venues_sorted.set_index('city'), on='city')

ohio_merged.head() # check the last columns!

In [ ]:
pd.DataFrame.to_csv(ohio_merged, "ohio_merge.csv")

# additional analysis were done through R

```{r}
library(tidyverse)
library(rvest)
library(ggmap)
library(jsonlite)
```

```{r}
url <- "https://en.wikipedia.org/wiki/List_of_cities_in_Ohio"

city <- read_html(url) %>%
  html_table(fill = TRUE)


city2 <- tibble(city[[2]]) %>%
  distinct(City, .keep_all = TRUE) %>% 
  select(1:2) %>%
  mutate(City = str_replace_all(City,"[:blank:]","+"))

lat_long <- tibble(city = as.character(), lat = as.character(), lon = as.character())

for (c in city2$City) {
  
open <- paste0("http://nominatim.openstreetmap.org/search?city=",c,"&state=ohio&format=json")


t <- fromJSON(open)[1,]

lat_long <- lat_long %>%
  add_row(city = c, lat = t$lat, lon = t$lon)

# Sys.sleep(sample(1:3))
}

write_csv(lat_long, path = "lat_long_ohio_city.csv")


```

```{r}
# add town summit county
url <- "https://council.summitoh.net/pages/Cities-Villages-and-Townships.html"

summitcounty <- read_html(url) %>%
  html_nodes(xpath =  "//*[@id=\"bodyTop\"]/main/div/div/div/div[3]/div[1]/table") %>%
  html_text()

summit_town <- str_split(summitcounty, pattern = "\r\n\t\t\t|\r\n")[[1]]  
summit_town <- str_replace(summit_town, pattern = "\t|\r\n\t", "")
summit_town <- summit_town[str_detect(summit_town, "Village|Township")]
summit_town <- str_replace_all(summit_town, " ", "+")
 
lat_long <- tibble(city = as.character(), lat = as.character(), lon = as.character())

for (c in summit_town) {
  
open <- paste0("http://nominatim.openstreetmap.org/search?city=",c,"&state=ohio&format=json")


t <- fromJSON(open)[1,]

lat_long <- lat_long %>%
  add_row(city = c, lat = t$lat, lon = t$lon)

# Sys.sleep(sample(1:3))
}

write_csv(lat_long, "lat_long_ohio_city.csv", append = TRUE)

```


```{r}
# let's see which is closer to home

df <- read_csv("ohio_venues.csv") %>%
  select(-1) 

df_cleaned <- df %>%
  filter(`Venue Category` == "Bakery")


library(geosphere)

home_lat <- #insert your home_lat here
home_lon <- #insert your home_lon here

df_home <- df %>%
  mutate(distance_from_home = map2_dbl(.x = `Neighborhood Latitude`, .y = `Neighborhood Longitude`, ~ distm(c(.x, .y),c(home_lat,home_lon), fun = distHaversine)))


df_home %>%
  distinct(Neighborhood, .keep_all = TRUE) %>%
  arrange(distance_from_home) %>%
  view()



df_home_dist <- df_home %>%
  select(city=Neighborhood,distance_from_home) %>%
  distinct()


```

```{r}
df_knn <- read_csv("ohio_merge.csv") %>%
  select(-1,-2) %>%
  left_join(df_home_dist, by = "city") %>%
  select(1:4,15,5:14)

library(tidyverse)

library(kableExtra)
df_knn_t <- df_knn %>%
  filter(`Cluster Labels` == 2) %>%
  filter(distance_from_home <= 50000) %>%
  pivot_longer(., cols = -c(1:5), names_to = "most", values_to = "category") %>%
  filter(category != "Bakery") %>%
  pivot_wider(., names_from = "most", values_from = "category") %>%
  drop_na() %>%
  arrange(distance_from_home) %>%
  filter(!str_detect(city, "zzz")) %>%
  mutate(city = str_replace(city, "\\+", " ")) %>%
  rename(City = city) %>%
  select(-c(2:5)) 

# ccreating table 
df_knn_t %>%
  kbl(caption = "Table 1: Cities in Cluster 2 and its Top 10 Businesses") %>%
  kable_classic(full_width = F, html_font = "Cambria")

# write_csv(df_knn_t, "c:/Users/dap3a/Downloads/cluster2.csv")

pop <- read_csv("cluster2_rent.csv") 

pop_mod <- pop %>%
  drop_na() %>%
  mutate(pop18_65_percent = 100 - pop5 - pop18 - pop65,
         pop18_65 = pop18_65_percent*0.01*0.001*pop,
         rent_sq = 900*rent/12 + (pop18_65/5*rent)/12,
         baked_goods = 100*5*4 + pop18_65*4,
         baked_profit = baked_goods*2,
         employee = ceiling((baked_goods/20)/100),
         profit = baked_profit - rent_sq
         ) %>%
  arrange(desc(pop18_65_percent),desc(median_income),desc(profit),desc(pop)) %>%
  select(City, `Total Population` = pop,`% Population of Age 18 through 65` = pop18_65_percent, `Median Income` = median_income,`Rent ($ per month)` = rent_sq, Profit = profit)


pop_mod %>%
  kbl(caption = "Table 2: Cities, Population, Rent and Profit Analysis") %>%
    kable_classic(full_width = F, html_font = "Cambria")

```

```{r}
library(tidycensus)

census_api_key("c53c84b926a847bf8cde9a5046a8d5d60431113b", install = TRUE)

readRenviron("~/.Renviron")

load_variables(2018, "acs1")

```

```{r}
library(leaflet)

df_knn %>%
  filter(`Cluster Labels` == 2) %>%
    filter(distance_from_home <= 50000) %>%
  pivot_longer(., cols = -c(1:5), names_to = "most", values_to = "category") %>%
  filter(category != "Bakery") %>%
  pivot_wider(., names_from = "most", values_from = "category") %>%
    filter(!str_detect(city, "zzz")) %>%
  drop_na() %>%
  select(1:3) %>%
  leaflet() %>%
  addTiles() %>%
  addMarkers()


```
